In [5]:
import pickle
import pandas as pd

# Simulated disaster chain event log
data = {
    "Timestamp": [
        "2023-07-01 10:00", "2023-07-01 15:00",
        "2023-07-02 08:00", "2023-07-04 12:00"
    ],
    "Location": ["Region A", "Region B", "Region A", "Region C"],
    "Disaster_Type": ["Earthquake", "Flood", "Landslide", "Heavy_Rain"],
    "Severity": ["High", "Medium", "Medium", "High"]
}

df = pd.DataFrame(data)
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df.sort_values("Timestamp", inplace=True)
df.reset_index(drop=True, inplace=True)
df


,Timestamp,Location,Disaster_Type,Severity
0,2023-07-01 10:00:00,Region A,Earthquake,High
1,2023-07-01 15:00:00,Region B,Flood,Medium
2,2023-07-02 08:00:00,Region A,Landslide,Medium
3,2023-07-04 12:00:00,Region C,Heavy_Rain,High


In [6]:
# Define chain rules: Primary → Secondary disaster within time (in hours)
disaster_chain_rules = {
    "Earthquake": {"next": "Landslide", "max_delay_hrs": 48},
    "Landslide": {"next": "Dam_Breach", "max_delay_hrs": 24},
    "Heavy_Rain": {"next": "Urban_Flood", "max_delay_hrs": 36}
}


In [7]:
from datetime import timedelta

alerts = []

for i, row in df.iterrows():
    current_disaster = row["Disaster_Type"]
    location = row["Location"]
    timestamp = row["Timestamp"]

    # If it's a primary event in chain rules
    if current_disaster in disaster_chain_rules:
        chain = disaster_chain_rules[current_disaster]
        next_disaster = chain["next"]
        time_window = timedelta(hours=chain["max_delay_hrs"])

        # Check if next disaster occurred in that window
        occurred = df[
            (df["Location"] == location) &
            (df["Disaster_Type"] == next_disaster) &
            (df["Timestamp"] > timestamp) &
            (df["Timestamp"] <= timestamp + time_window)
        ]

        if occurred.empty:
            alerts.append({
                "Alert": f"⚠️ Likely {next_disaster} after {current_disaster} in {location}",
                "Action": f"Deploy resources and monitor until {timestamp + time_window}"
            })

# Show alerts
pd.DataFrame(alerts)


,Alert,Action
0,⚠️ Likely Dam_Breach after Landslide in Region A,Deploy resources and monitor until 2023-07-03 ...
1,⚠️ Likely Urban_Flood after Heavy_Rain in Regi...,Deploy resources and monitor until 2023-07-06 ...
